In [19]:
import torch
from torchvision.datasets import OxfordIIITPet
from torchvision import transforms
from torchvision import models
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
import wandb
from tqdm.auto import tqdm

c:\Users\nicol\anaconda3\envs\cv_clf_xai\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:
device = ('cuda' if torch.cuda.is_available() else 'cpu')

In [21]:
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],   # Normalization of ImageSet (necessary if we use a model pre-trained on ImageSet, such as ResNet-18)
                         std=[0.229, 0.224, 0.225])
])
test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                         std=[0.229, 0.224, 0.225])
])

train_data = OxfordIIITPet(
    root="../data",
    split="trainval",
    transform=train_transform,
    download=True
)
test_data = OxfordIIITPet(
    root="../data",
    split="test",
    transform=test_transform,
    download=True
)
print(f"Training samples: {len(train_data)}")
print(f"Test samples: {len(test_data)}")
print(f'Size of the images: {train_data[0][0].shape}')

Training samples: 3680
Test samples: 3669
Size of the images: torch.Size([3, 224, 224])


In [22]:
class_names = train_data.classes
label_to_breed = {i:name for i, name in enumerate(class_names)}  # Map between the labels and the names of the pet breeds

CAT_BREEDS = {
    "Abyssinian", "Bengal", "Birman", "Bombay", "British_Shorthair", 
    "Egyptian_Mau", "Maine_Coon", "Persian", "Ragdoll", "Russian_Blue", 
    "Siamese", "Sphynx"
}

label_to_species = {i: ('cat' if name in CAT_BREEDS else 'dog')   # Map between the labels and the species of the animal (cat or dog)
                    for i, name in enumerate(class_names)}

In [23]:
def load_model(model, dropout_p=0.5):
    """
    Load a pre-trained model and set it up for training:
    replace the last fully-connected layer and freeze
    the rest of the model.
    """
    # Freeze all the layers in the base model
    for param in model.parameters():
        param.requires_grad = False

    # Get the number of input features for the classifier
    num_ftrs = model.fc.in_features
    num_classes = len(class_names)

    # Create a new fully-connected layer for our new classes
    model.fc = nn.Sequential(
        nn.Dropout(p=dropout_p),
        nn.Linear(num_ftrs, num_classes)
    ) 

    # Move model to device
    model.to(device)
    
    return model

In [ ]:
def fine_tune(model, dataloaders, loss_fn, optimizer, epochs=10, device='cuda'):
    """
    """ 
    # Create a dictionary to store training history
    history = {'train_loss': [], 'val_loss': [], 'val_acc': []}
    train_dataloader, test_dataloader = dataloaders

    # Loop through epochs
    for epoch in range(epochs):
        ### Training Phase ###
        model.train()
        
        train_loss = 0.0
        for X, y in tqdm(train_dataloader, desc=f'Epoch {epoch+1} Training', leave=False):
            # Move data to target device
            X, y = X.to(device), y.to(device)

            # Forward pass
            y_pred = model(X)

            # Calculate loss
            loss = loss_fn(y_pred, y)
            train_loss += loss.item()

            # Optimizer zero grad
            optimizer.zero_grad()

            # Loss backward
            loss.backward()

            # Optimizer step
            optimizer.step()

        # Calculate average training loss for the epoch
        train_loss /= len(train_dataloader)

        ### Validation Phase ###
        model.eval() # Set model to evaluation mode
        
        val_loss, val_acc = 0.0, 0.0
        with torch.inference_mode():
            for X, y in test_dataloader:
                X, y = X.to(device), y.to(device)
                
                # Forward pass
                val_pred = model(X)
                
                # Calculate loss and accuracy
                val_loss += loss_fn(val_pred, y).item()
                val_acc += (val_pred.argmax(dim=1) == y).sum().item()

        # Calculate metrics over the whole validation set
        val_loss /= len(test_dataloader)
        val_acc /= len(test_dataloader.dataset)

        # Print progress
        print(f"Epoch: {epoch+1} | "
              f"Train loss: {train_loss:.4f} | "
              f"Val loss: {val_loss:.4f} | "
              f"Val acc: {val_acc:.4f}")

        # Store history
        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)
        
        wandb.log({
            "train_loss": train_loss,
            "val_loss": val_loss,
            "val_acc": val_acc
        })
        
    return history
    
def fine_tune_full(model, dataloaders, loss_fn, optimizers, epochs_head=5, epochs_full=5, device='cuda'):
    """
    """
    optimizer_head, optimizer_full = optimizers
    
    # Train the head of the model
    print('\nTraining model\'s head...')
    history_head = fine_tune(model, dataloaders, loss_fn, optimizer_head, epochs=epochs_head, device=device)
    
    # Unfreeze all layers
    for param in model.parameters():
        param.requires_grad = True
        
    print('Training full model...')
    # Train the full model
    history_full = fine_tune(model, dataloaders, loss_fn, optimizer_full, epochs=epochs_full, device=device)
    print('Finished Training')
    
    # Join the histories
    history = {'train_loss': history_head['train_loss'] + history_full['train_loss'], 
               'val_loss': history_head['val_loss'] + history_full['val_loss'], 
               'val_acc': history_head['val_acc'] + history_full['val_acc']}
    
    return history

In [25]:
# Set up ResNet-18
model_resnet18 =  models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
model_resnet18 = load_model(model_resnet18)
# Set up ResNet-34
model_resnet34 =  models.resnet34(weights=models.ResNet34_Weights.DEFAULT)
model_resnet34 = load_model(model_resnet34)

In [ ]:
# Config for W&B sweep to find optimal hyperparameters
sweep_config = {
    'entity': 'nicdeluc-learning',
    'project': 'pet-breed-classification', 
    'method': 'bayes',           # The search strategy: bayes, random, or grid
    'metric': {
        'name': 'val_acc' ,       
        'goal': 'maximize'
    },
    'parameters': {
        'architecture': {
            'values': ['resnet18', 'resnet34']
        },
        'lr_head': {
            'distribution': 'log_uniform_values',
            'min': 2e-4,
            'max': 2e-3
        },
        'lr_full': {
            'distribution': 'log_uniform_values',
            'min': 2e-5,
            'max': 1e-4
        },
        'weight_decay_full': {
            'distribution': 'log_uniform_values',
            'min': 1e-4,
            'max': 1e-2
        },
        'dropout_p': {
            'values': [0.25, 0.5, 0.75]
        },
        # 'epochs_head': {
        #     'distribution': 'int_uniform',
        #     'min': 5,
        #     'max': 15
        # },
        # 'epochs_full': {
        #     'distribution': 'int_uniform',
        #     'min': 10,
        #     'max': 30
        # }
    }  
}

# Sweep function for ResNet-18
def sweep_fn():
    """
    """
    with wandb.init():
        config = wandb.config
        
        train_dataloader = DataLoader(train_data, batch_size=64, shuffle=True)
        test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)
        
        if config.architecture == "resnet18":
            model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
            model = load_model(model, dropout_p=config.dropout_p) 
        elif config.architecture == "resnet34":
            model = models.resnet34(weights=models.ResNet34_Weights.DEFAULT)
            model = load_model(model, dropout_p=config.dropout_p) 
            
        loss_fn = nn.CrossEntropyLoss()
        
        optim_head = optim.Adam(model.fc.parameters(), 
                                lr=config.lr_head)
        optim_full = optim.Adam(model.parameters(), 
                                lr=config.lr_full, 
                                weight_decay=config.weight_decay_full)
        
        history = fine_tune_full(model, 
                                 dataloaders = (train_dataloader, test_dataloader), 
                                 loss_fn = loss_fn, 
                                 optimizers = (optim_head, optim_full), 
                                 epochs_head = 10, 
                                 epochs_full = 20)
        
    return history
        

In [27]:
# Initialize the sweep
sweep_id = wandb.sweep(sweep_config, )

# Run the agent for the specified number of times
wandb.agent(sweep_id, function=sweep_fn, count=10)

Create sweep with ID: fhbxuj24
Sweep URL: https://wandb.ai/nicdeluc-learning/pet-breed-classification/sweeps/fhbxuj24


wandb: Agent Starting Run: fvkzk2w7 with config:
wandb: 	architecture: resnet18
wandb: 	dropout_p: 0.25
wandb: 	lr_full: 0.00014028010210727517
wandb: 	lr_head: 0.0017624787761338124
wandb: 	weight_decay_full: 0.0004088532703021322


Training model's head...


Epoch: 1 | Train loss: 2.3687 | Val loss: 1.0362 | Val acc: 0.7634


Epoch: 2 | Train loss: 1.2502 | Val loss: 0.6971 | Val acc: 0.8182


...

Epoch: 17 | Train loss: 0.5090 | Val loss: 0.3189 | Val acc: 0.9065


Epoch: 18 | Train loss: 0.5138 | Val loss: 0.3109 | Val acc: 0.9052


Epoch: 19 | Train loss: 0.4975 | Val loss: 0.3147 | Val acc: 0.9062


Epoch: 20 | Train loss: 0.5061 | Val loss: 0.3062 | Val acc: 0.9081
Finished Training


train_loss,█▆▅▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_acc,▁▅▆▆▇▇▇▇▇▇▇███████████████████
val_loss,█▆▄▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss,0.50611
val_acc,0.90815
val_loss,0.30623


In [28]:
# TODO:
# - Implement early stopping